In [1]:
#importing packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None

In [2]:
#reading data
data = pd.read_csv('./datasets/kc_house_data.csv')

#formatting date
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

### First question

In [24]:
#grouping by median per zipcode
df1 = data[['price', 'zipcode']].groupby('zipcode').median().reset_index()

In [25]:
#merging result
data1 = pd.merge(data, df1, on='zipcode', how='inner')[['id', 'zipcode', 'price_x', 'price_y', 'condition']]

In [26]:
#recommendation
data1['recommendation'] = np.nan

for i in range(len(data1)):
    if (data1.loc[i, 'price_x'] <= data1.loc[i, 'price_y']) & (data1.loc [i, 'condition'] >= 4):
        data1.loc[i, 'recommendation'] = 'buy'
    else:
        data1.loc[i, 'recommendation'] = 'dont buy'

In [27]:
#filtering data
data1 = data1[data1['recommendation'] == 'buy'].reset_index(drop=True)

In [30]:
#formatting price
for i in range(len(data1)):
    data1.loc[i, 'price_x'] = "${:0,.2f}".format(data1.loc[i, 'price_x'])
    data1.loc[i, 'price_y'] = "${:0,.2f}".format(data1.loc[i, 'price_y'])

In [46]:
#formatting condition
for i in range(len(data1)):
    if data1.loc[i, 'condition'] <= 1:
        data1.loc[i, 'condition'] = 'bad'
    elif data1.loc[i, 'condition'] <=3:
        data1.loc[i, 'condition'] = 'regular'
    elif data1.loc[i, 'condition'] <=4:
        data1.loc[i, 'condition'] = 'good'
    else:
        data1.loc[i, 'condition'] = 'great'

In [47]:
#renaming columns
data1.columns = ['ID', 'Zip Code', 'Price', 'Price Median Zipcode', 'Condition', 'Recommendation']

In [10]:
#saving dataframe to csv
data1_report = data1
data1_report.to_csv('Recommendation_Report_Buy.csv', index=False)

### Second question

In [65]:
#extracting month
data['month'] = pd.to_datetime(data['date']).dt.strftime('%m').astype(np.int64)
#creating column season
data['season'] = data['month'].apply(lambda x: 'summer' if (x >= 6) & (x <= 8)
                                     else 'autumn' if (x >= 9) & (x <= 11)
                                     else 'spring' if (x >= 3) & (x <= 5)
                                     else 'winter')

In [66]:
#grouping by median per zipcode and season
df2 = data[['price', 'zipcode', 'season']].groupby(['zipcode', 'season']).median().reset_index()

In [67]:
#merging result
df3 = pd.merge(data, df2, on=['zipcode', 'season'], how='inner')[['id', 'zipcode','season', 'price_y', 'price_x', 'condition']]

In [68]:
#formatting condition
for i in range(len(df3)):
    if (df3.loc[i, 'price_x'] <= df3.loc[i, 'price_y']) & (df3.loc [i, 'condition'] >= 5):
        df3.loc[i, 'recommendation'] = 'buy'
    else:
        df3.loc[i, 'recommendation'] = 'dont buy'

In [69]:
#filtering data
df3 = df3[df3['recommendation'] == 'buy'].reset_index(drop=True)

In [70]:
#formatting condition
for i in range(len(df3)):
    if df3.loc[i, 'condition'] <= 1:
        df3.loc[i, 'condition'] = 'bad'
    elif df3.loc[i, 'condition'] <=3:
        df3.loc[i, 'condition'] = 'regular'
    elif df3.loc[i, 'condition'] <=4:
        df3.loc[i, 'condition'] = 'good'    
    else:
        df3.loc[i, 'condition'] = 'great'

In [71]:
#creating columns sell price and profit
df3['sell_price'] = np.nan
df3['profit'] = np.nan
#filling sell price column
for i in range(len(df3)):
    df3.loc[i, 'sell_price'] = df3.loc[i, 'price_x'] * 1.3
#filling profit column
for i in range(len(df3)):
    df3.loc[i, 'profit'] = df3.loc[i, 'sell_price'] - df3.loc[i, 'price_x']

In [75]:
#filtering data
df3 = df3[df3['sell_price'] <= df3['price_y']].reset_index(drop=True)

In [79]:
#renaming columns
df3.columns = ['ID', 'Zip Code', 'Season', 'Avg Price', 'Buy Price', 'Condition', 'Recommendation', 'Sell Price', 'Profit']

In [16]:
#saving dataset to csv
df3_report = df3
df3_report.to_csv('Recommendation_Report_Sell.csv', index=False)

### Pergunta 3

In [83]:
#creating new dataframe
data3 = df3.copy()

In [86]:
#creating new feature
data3 = data3.assign(Dif_Avg_Sell = lambda x: (x['Avg Price'] - x['Sell Price']))

In [102]:
#summing total investment and profit
invest = data3['Buy Price'].sum()
profit = data3['Profit'].sum()

In [106]:
#creating new dataframe grouped by zip code and season / count
data3count = data3[['ID', 'Zip Code', 'Season']].groupby(['Zip Code', 'Season']).count().reset_index()

In [107]:
#creating new dataframe grouped by zip code and season / sum
data3sum = data3[['Profit', 'Season', 'Zip Code']].groupby(['Zip Code', 'Season']).sum().reset_index()

In [109]:
#merging two dataframes
data3 = pd.merge(data3count, data3sum, on=['Zip Code', 'Season'], how='inner')

In [112]:
#renaming columns
data3.columns = ['Zip Code', 'Season', 'Houses', 'Total Profit']

### Hypothesis

##### H1 - Waterfront houses are in average 30% more expensive than others

In [4]:
#creating new dataframe
dfh1 = data[['id', 'price', 'waterfront']]

#calculating mean
mean_price = dfh1[dfh1['waterfront'] == 0]['price'].mean()
mean_price_wf = dfh1[dfh1['waterfront'] == 1]['price'].mean()

#calculating difference 
dif_wf_notwf = (mean_price_wf - mean_price)/mean_price
dif_wf_perc = "{:.0%}".format(dif_wf_notwf)

#printing result
if dif_wf_notwf > 0.3:
    print('Hipótese validada! Casas com vista para água são em média {} mais caras que as outras.'.format(dif_wf_perc))
else:
    print('Hipótese invalidada! A diferença média de preço é de {}.'.format(dif_wf_perc))

Hipótese validada! Casas com vista para água são em média 213% mais caras que as outras.


##### H2 - Houses built before 1955 are in average 50% cheaper than the ones built after 1955

In [139]:
#creating new dataset
dfh2 = data[['id', 'price', 'yr_built']]

#calculating mean
mean_price = dfh2['price'].mean()
mean_price_1955back = dfh2[dfh2['yr_built'] < 1955]['price'].mean()

#calculating difference 
dif_1955_forw = (mean_price_1955back - mean_price)/mean_price
dif_1955_perc = "{:.0%}".format(dif_1955_forw)

#printing result
if dif_1955_forw <= -0.5:
    print('Hipótese validada! Casas contruídas antes de 1955 são em média {} mais baratas que as demais.'.format(dif_1955_perc))
else:
    print('Hipótese invalidada! A diferença média de preço é de {}.'.format(dif_1955_perc))


Hipótese invalidada! A diferença média de preço é de -1%.


##### H3 - Houses with basement are in average 40% bigger than others

In [140]:
#creating new dataframe
dfh3 = data[['id', 'sqft_lot', 'sqft_basement']]

##calculating mean
mean_size_withbasement = dfh3[dfh3['sqft_basement'] != 0]['sqft_lot'].mean()
mean_size_nobasement = dfh3[dfh3['sqft_basement'] == 0]['sqft_lot'].mean()

#calculating difference 
dif_size = (mean_size_withbasement-mean_size_nobasement)/mean_size_nobasement
dif_size_perc = "{:.0%}".format(dif_size)

#printing answer
if dif_size > 0.4:
    print('Hipótese validada! Casas sem porão são em média {} maiores que as demais.'.format(dif_size_perc))
else:
    print('Hipótese invalidada! A diferença média de tamanho é de {}.'.format(dif_size_perc))

Hipótese invalidada! A diferença média de tamanho é de -18%.


##### H4 - House prices grew up 10% YoY

In [12]:
#creating filtered dataset
dfh4 = data[['id', 'price', 'date']]
dfh4['year'] = dfh4['date'].dt.year

#mean price by year
price_by_year = dfh4[['price', 'year']].groupby('year').mean().reset_index()

#calculating difference
price_dif = (price_by_year.iloc[1,1] - price_by_year.iloc[0,1])/price_by_year.iloc[0,1]
price_dif_per = "{:.0%}".format(price_dif)

#printing result
if price_dif > 0.1:
    print('Hipótese validada! Cresimento médio de {} YoY'.format(price_dif_per))
else:
    print('Hipótese invalidada! Cresimento médio de {} YoY'.format(price_dif_per))

Hipótese invalidada! Cresimento médio de 1% YoY


##### H5 - 3 bathrooms houses grew up price in average 15% MoM 

In [15]:
#creating filtered dataframe
dfh5 = data[data['bathrooms'] == 3][['id', 'date', 'price']].reset_index(drop=True)
dfh5['year_month'] = dfh5['date'].dt.strftime('%Y-%m')

#mean price by year/month
price_by_year_month = dfh5[['price', 'year_month']].groupby('year_month').mean().reset_index()

#calculanting difference
price_by_year_month['price_dif'] = price_by_year_month.price.diff()
price_by_year_month['price_dif_percent'] = price_by_year_month['price'].pct_change()
price_by_year_month['price_dif_perc'] = price_by_year_month['price'].pct_change().apply(lambda x: "{:.2%}".format(x))

#calculating mean difference
mean_dif = price_by_year_month['price_dif_percent'].mean()
mean_dif_perc = "{:.2%}".format(price_by_year_month['price_dif_percent'].mean())

#printing result
if mean_dif > 0.15:
    print('Hipótese validada! Cresimento médio de {} MoM'.format(mean_dif_perc))
else:
    print('Hipótese invalidada! Cresimento médio de {} MoM na média.'.format(mean_dif_perc))

Hipótese invalidada! Cresimento médio de 0.23% MoM na média.


In [16]:
dfh5.dtypes

id                     int64
date          datetime64[ns]
price                float64
year_month            object
dtype: object

##### H6 - Renovated houses are in average 15% more expansive than others

In [143]:
#creating filtered dataset
dfh6 = data[['id', 'price', 'yr_renovated']]

#mean price
mean_price_renov = dfh6[dfh6['yr_renovated'] == 0]['price'].mean()
mean_price_notrenov = dfh6[dfh6['yr_renovated'] != 0]['price'].mean()

#calculating difference
mean_price_dif = (mean_price_notrenov - mean_price_renov)/mean_price_notrenov
mean_price_dif_perc = "{:.2%}".format(mean_price_dif)

#printing result
if mean_price_dif >= 0.15:
    print('Hipótese validada! Casas reformadas são em média {} mais caras que outras.'.format(mean_price_dif_perc))
else:
    print('Hipótese invalidada! Diferença é de {}'.format(mean_price_dif_perc))

Hipótese validada! Casas reformadas são em média 30.25% mais caras que outras.


##### H7 - Waterfront houses are in average 10% more expansive in summer

In [8]:
datadfh7 = data.copy()

#extracting month
datadfh7['month'] = pd.to_datetime(datadfh7['date']).dt.strftime('%m').astype(np.int64)
#creating column season
datadfh7['season'] = datadfh7['month'].apply(lambda x: 'summer' if (x >= 6) & (x <= 8)
                                    else 'autumn' if (x >= 9) & (x <= 11)
                                    else 'spring' if (x >= 3) & (x <= 5)
                                    else 'winter')

#creating filtered dataframe
dfh7 = datadfh7[datadfh7['waterfront'] == 1][['id', 'price', 'waterfront', 'season']]

#mean price
mean_price_summer = dfh7[dfh7['season'] == 'summer']['price'].mean()
mean_price_not_summer = dfh7[dfh7['season'] != 'summer']['price'].mean()

#calculating difference
price_dif = (mean_price_summer-mean_price_not_summer)/mean_price_not_summer
price_dif_perc = '{:.2%}'.format(price_dif)

#printing result
if price_dif >= 0.1:
    print('Hipótese validada! Diferença de {}.'.format(price_dif_perc))
else:
    print('Hipótese invalidada! Diferença é de  {}.'.format(price_dif_perc))

Hipótese validada! Diferença de 14.26%.


##### H8 - Houses with more than one floor are in average 30% more expansive than others

In [145]:
#creating filtered dataset
dfh8 = data[['id', 'price', 'floors']]

#mean price
mean_price_notonefloor = dfh8[dfh8['floors'] > 1]['price'].mean()
mean_price_onefloor = dfh8[dfh8['floors'] <= 1]['price'].mean()

#calculating difference
dif_price_floor = (mean_price_notonefloor-mean_price_onefloor)/mean_price_notonefloor
dif_price_floor_per = '{:.2%}'.format(dif_price_floor)

#printing result
if dif_price_floor >0.3:
    print('Hipótese validada! Diferença de {}.'.format(dif_price_floor_per))
else:
    print('Hipótese invalidada! Diferença é de {}.'.format(dif_price_floor_per))

Hipótese validada! Diferença de 30.45%.


##### H9 - More than 70% of houses built after than the year 2000 are in average 30% smaller than the ones built before 2000

In [146]:
#converting sqft to sqmt
def sqft_to_sqmt (x):
    return x * 0.09290304

In [147]:
#creating filtered dataframe
dfh9 = data[['id', 'sqft_lot', 'yr_built']]
dfh9new = dfh9

#mean size
mean_sqmt_before2k = sqft_to_sqmt(dfh9[dfh9['yr_built'] < 2000]['sqft_lot'].mean())
mean_sqmt_after2k = sqft_to_sqmt(dfh9[dfh9['yr_built'] >= 2000]['sqft_lot'].mean())

dfh9after2k = dfh9[dfh9['yr_built'] >= 2000].reset_index(drop=True)
dfh9after2k['sqmt_before2k'] = mean_sqmt_before2k
dfh9after2k['sqmt_after2k'] = dfh9after2k['sqft_lot'].apply(lambda x: (sqft_to_sqmt(x)))

dfh9after2k['dif_sqmt'] = np.nan
for i in range(len(dfh9after2k)):
    dfh9after2k.loc[i, 'dif_sqmt'] = (dfh9after2k.loc[i, 'sqmt_before2k']- dfh9after2k.loc[i, 'sqmt_after2k'])/dfh9after2k.loc[i, 'sqmt_before2k']
    
houses_smaller_mean = dfh9after2k[dfh9after2k['dif_sqmt'] >= -0.3]['id'].count()
houses_total = dfh9after2k['id'].count()

housessmaller_vs_totalhouses = houses_smaller_mean/houses_total
housessmaller_vs_totalhouses_perc = '{:.2%}'.format(housessmaller_vs_totalhouses)

#printing result
if housessmaller_vs_totalhouses >= 0.7:
    print('Hipótese validada! {} das casas construídas depois dos anos 2000 são pelo menos 30% ' \
          'menores que as demais'.format(housessmaller_vs_totalhouses_perc))
else:
    print('Hipótese invalidada! Somente {} das casas construídas depois dos anos 2000 são pelo menos 30% ' \
          'menores que as demais.'.format(housessmaller_vs_totalhouses_perc))

Hipótese validada! 94.35% das casas construídas depois dos anos 2000 são pelo menos 30% menores que as demais


##### H10 - More than 80% of houses with 4 or more bedrooms and with less than 3 bathrooms are below the region average price

In [148]:
#converting sqft to sqmt
dfh10 = data[['id', 'price', 'zipcode', 'bedrooms', 'bathrooms']]
dfh10pricezipcode = dfh10[['price', 'zipcode']].groupby('zipcode').mean().reset_index()
dfh10 = pd.merge(dfh10, dfh10pricezipcode, on='zipcode', how='inner')

dfh10['dif_price'] = np.nan

for i in range(len(dfh10)):
    dfh10.loc[i, 'dif_price'] = (dfh10.loc[i, 'price_x']-
                                 dfh10.loc[i, 'price_y'])/dfh10.loc[i, 'price_y']

dfh10new = dfh10[(dfh10['bedrooms'] >= 4)&(dfh10['bathrooms'] < 3)]
housesbelowmean = dfh10new[dfh10new['dif_price'] < 0]['id'].count()
housestotal = dfh10new['id'].count()

housesbellow_vs_totalhouses = housesbelowmean/housestotal
housesbellow_vs_totalhouses_per = '{:.2%}'.format(housesbellow_vs_totalhouses)

#printing result
if housesbellow_vs_totalhouses > 0.8:
    print ('Hipótese validada! {} dessas casas estão abaixo do preço médio da região.'.format(housesbellow_vs_totalhouses_per))
else:
    print ('Hipótese invalidada! Somente {} dessas casas estão abaixo do preço médio da região.'.format(housesbellow_vs_totalhouses_per))

Hipótese invalidada! Somente 49.82% dessas casas estão abaixo do preço médio da região.
